In [1]:
from PIL import Image
import os
import json
from tqdm import tqdm

In [2]:
test_anno = "F:/UNI/Proposal Skripsi/deepfashion2_dataset/validation/annos"
test_img = "F:/UNI/Proposal Skripsi/deepfashion2_dataset/validation/image"
test_yolo_anno = "F:/Skripsi/deepfashion2_dataset_filter/val/yolo_anno"

In [4]:
def convert_to_yolo_format(annotation, image_width, image_height):
    bounding_box = annotation['bounding_box']
    category_id = annotation['category_id']
    category_name = annotation['category_name']
    
    x_min, y_min, x_max, y_max = bounding_box
    
    x_center = ((x_max + x_min) / 2) / image_width
    y_center = ((y_max + y_min) / 2) / image_height
    width = (x_max - x_min) / image_width
    height = (y_max - y_min) / image_height
    
    return f"{category_id} {x_center} {y_center} {width} {height}"

def process_directory(src_anno_dir, dest_dir, img_dir):
    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)
    
    # Dictionary to store unique category_id and category_name pairs
    category_dict = {}
    
    for filename in tqdm(os.listdir(src_anno_dir), desc=f"Processing {src_anno_dir}"):
        if filename.endswith('.json'):
            with open(os.path.join(src_anno_dir, filename), 'r') as f:
                data = json.load(f)
            
            for key, value in data.items():
                if 'item' in key:
                    # Update the dictionary with unique category_id and category_name pairs
                    category_dict[value['category_id']] = value['category_name']
                    
                    img_filename = os.path.splitext(filename)[0] + '.jpg'
                    img_path = os.path.join(img_dir, img_filename)
                
                    img = Image.open(img_path)
                    image_width, image_height = img.size
                    img.close()
                
                    yolo_line = convert_to_yolo_format(value, image_width, image_height)
                    yolo_lines = [yolo_line]
                
                    txt_filename = os.path.splitext(filename)[0] + '.txt'
                    with open(os.path.join(dest_dir, txt_filename), 'w') as f:
                        f.write('\n'.join(yolo_lines))
    
    # Display the category_id and category_name pairs in the desired format
    for category_id, category_name in sorted(category_dict.items()):
        print(f"{category_id}: {category_name}")

#Execution
process_directory(test_anno, test_yolo_anno, test_img)

Processing F:/UNI/Proposal Skripsi/deepfashion2_dataset/validation/annos: 100%|██| 32153/32153 [09:31<00:00, 56.27it/s]

1: short sleeve top
2: long sleeve top
3: short sleeve outwear
4: long sleeve outwear
5: vest
6: sling
7: shorts
8: trousers
9: skirt
10: short sleeve dress
11: long sleeve dress
12: vest dress
13: sling dress
